In [1]:
import numpy as np
import pylab as pl
from scipy.stats import t, norm, f
import operator


In [2]:
def t_test(sample1, sample2, confidence):
    print ('Medias:', np.mean(sample1), np.mean(sample2))
    #print ('Varianzas:', np.var(sample1, ddof = 1), np.var(sample2, ddof = 1))
    #print ('Muestra:', len(sample1), len(sample2))
    
    if len(sample1) <= 1 or len(sample2) <= 1:
        return "-"
    ddof1 = 1
    ddof2 = 1
    if len(sample1) == 1:
        ddof1 = 0
    if len(sample2) == 1:
        ddof2 = 0

    significance = 1 - confidence
    if var_test(sample1, sample2, confidence):
        sp = (((len(sample1)-1)*np.var(sample1, ddof = ddof1)+(len(sample2)-1)*np.var(sample2, ddof = ddof2))/(len(sample1)+len(sample2)-2))**0.5
        #print(sp)
        z = (abs(np.mean(sample1) - np.mean(sample2)))/(sp * (1/len(sample1) + 1/len(sample2))**0.5)
        df = len(sample1) + len(sample2) - 2
        t_statistic = t.ppf(significance/2, df)
        ic_min = (np.mean(sample1) - np.mean(sample2)) + t_statistic * (1/len(sample1) + 1/len(sample2))**0.5 * sp
        ic_max = (np.mean(sample1) - np.mean(sample2)) - t_statistic * (1/len(sample1) + 1/len(sample2))**0.5 * sp
        #print(t.ppf(significance/2, df) * (1/len(sample1) + 1/len(sample2))**0.5 * sp)
    else:
        sp = (np.var(sample1, ddof = ddof1)/len(sample1) + np.var(sample2, ddof = ddof2)/len(sample2))**0.5
        z = (abs(np.mean(sample1) - np.mean(sample2)))/sp
        #df = ((np.var(sample1)/len(sample1) + np.var(sample2)/len/sample2)**2)/(((np.var(sample1)/len(sample1))**2)/(len(sample1)-1) + ((np.var(sample2)/len(sample2))**2)/(len(sample2)-1))
        df = int((np.var(sample1, ddof = ddof1)/len(sample1) + np.var(sample2, ddof = ddof2)/len(sample2))**2/(((np.var(sample1, ddof = ddof1)/len(sample1))**2)/(len(sample1)+1) + ((np.var(sample2, ddof = ddof2)/len(sample2))**2)/(len(sample2)+1))) - 2
        #print((np.var(sample1, ddof = 1)/len(sample1) + np.var(sample2, ddof = 1))
        t_statistic = t.ppf(significance/2, df)
        ic_min = (np.mean(sample1) - np.mean(sample2)) + t_statistic * sp
        ic_max = (np.mean(sample1) - np.mean(sample2)) - t_statistic * sp
        #print(t.ppf(significance/2, df) * sp)
    
    p_value = 1 - t.cdf(z, df)
    return str(round(100*p_value,3)) + '%'


In [3]:
def var_test(sample1, sample2, confidence):
    if np.var(sample1, ddof = 1) > np.var(sample2, ddof = 1):
        s1 = np.var(sample1, ddof = 1)
        s2 = np.var(sample2, ddof = 1)
        n1 = len(sample1)
        n2 = len(sample2)
    else:
        s1 = np.var(sample2, ddof = 1)
        s2 = np.var(sample1, ddof = 1)
        n1 = len(sample2)
        n2 = len(sample1)
    z = (1-confidence)/2
    ic_min = s1/s2 * f.ppf(z, n2-1, n1-1)
    ic_max = s1/s2 * f.ppf(1-z, n2-1, n1-1)
    if ic_min < 1 < ic_max:
        #print ('same vars')
        return True
    else:
        #print ('different vars')
        return False

In [7]:
def detailed_adherence_test(clusters):
    patient_r = open('Reports/Explicit Derivation/Nuevos Clusters/patients.csv', 'r')
    lista_p = []
    patient_r.readline()
    for line in patient_r:
        lista_p.append(line.strip())
    patient_r.close()
    
    writer = open('Reports/Explicit Derivation/Adherence/Detailed/role_detailed_p_values.csv', 'w')
    writer2 = open('Reports/Explicit Derivation/Adherence/Detailed/role_detailed_cluster_frequency.csv', 'w')
    writer.write('MEDICO->MEDICO\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0

    ##MEDICO -> MEDICO
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            print(reader.readline().split(';')[2])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[2]) > 0):
                    s1.append(float(lista[1])/float(lista[2]))
                
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            reader.readline()
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[2]) > 0):
                    s2.append(float(lista[1])/float(lista[2]))

            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) - 2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')


        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')
    
    
    
    writer.write('MEDICO->ENFERMERA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0

    ##MEDICO -> ENFERMERA
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            print(reader.readline().split(';')[4])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[4]) > 0):
                    s1.append(float(lista[3])/float(lista[4]))

            reader.close()
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            reader.readline()
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[4]) > 0):
                    s2.append(float(lista[3])/float(lista[4]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n') 


    writer.write('MEDICO->NUTRICIONISTA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    ##MEDICO -> NUTRICIONISTA
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[6]) > 0):
                    s1.append(float(lista[5])/float(lista[6]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[6])
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[6]) > 0):
                    s2.append(float(lista[5])/float(lista[6]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')

        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')        

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n') 


    ##ENFERMERA -> MEDICO
    writer.write('ENFERMERA->MEDICO\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[8]) > 0):
                    s1.append(float(lista[7])/float(lista[8]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[8])
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[8]) > 0):
                    s2.append(float(lista[7])/float(lista[8]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')

    

    ##ENFERMERA -> ENFERMERA
    writer.write('ENFERMERA->ENFERMERA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[10]) > 0):
                    s1.append(float(lista[9])/float(lista[10]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[10])
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[10]) > 0):
                    s2.append(float(lista[9])/float(lista[10]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')

    ##ENFERMERA -> NUTRICIONISTA
    writer.write('ENFERMERA->NUTRICIONISTA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[12]) > 0):
                    s1.append(float(lista[11])/float(lista[12]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[12])
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[12]) > 0):
                    s2.append(float(lista[11])/float(lista[12]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')

    ##NUTRICIONISTA -> MEDICO
    writer.write('NUTRICIONISTA->MEDICO\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[14]) > 0):
                    s1.append(float(lista[13])/float(lista[14]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[14])
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[14]) > 0):
                    s2.append(float(lista[13])/float(lista[14]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')


    ##NUTRICIONISTA -> ENFERMERA
    writer.write('NUTRICIONISTA->ENFERMERA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[16]) > 0):
                    s1.append(float(lista[15])/float(lista[16]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[16])
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[16]) > 0):
                    s2.append(float(lista[15])/float(lista[16]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')

     
    ##NUTRICIONISTA -> NUTRICIONISTA
    writer.write('NUTRICIONISTA->NUTRICIONISTA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.strip().split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[18]) > 0):
                    s1.append(float(lista[17])/float(lista[18]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[18])
            s2 = []
            for line in reader:
                lista = line.strip().split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[18]) > 0):
                    s2.append(float(lista[17])/float(lista[18]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')

    writer2.close()
    writer.close()


In [8]:
def detailed_adherence_test_from_to(clusters):
    
    patient_r = open('Reports/Explicit Derivation/Nuevos Clusters/patients.csv', 'r')
    lista_p = []
    patient_r.readline()
    for line in patient_r:
        lista_p.append(line.strip())
    patient_r.close()
    
    writer = open('Reports/Explicit Derivation/Adherence/FromTo/role_detailed_p_values.csv', 'w')
    writer2 = open('Reports/Explicit Derivation/Adherence/FromTo/role_detailed_cluster_frequency.csv', 'w')
    writer.write('MEDICO->*\n;')
    
    ##MEDICO -> *
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0

    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            print(reader.readline().split(';')[2])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[2]) > 0):
                    s1.append(float(lista[1])/float(lista[2]))
                
                if (float(lista[4]) > 0):
                    s1.append(float(lista[3])/float(lista[4]))
                
                if (float(lista[6]) > 0):
                    s1.append(float(lista[5])/float(lista[6]))
                
                
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            reader.readline()
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[2]) > 0):
                    s2.append(float(lista[1])/float(lista[2]))
                    
                if (float(lista[4]) > 0):
                    s2.append(float(lista[3])/float(lista[4]))

                if (float(lista[6]) > 0):
                    s2.append(float(lista[5])/float(lista[6]))

            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) - 2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')


        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')
    
    
    ##ENFERMERA -> *
    writer.write('ENFERMERA->*\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0

    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            print(reader.readline().split(';')[4])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[8]) > 0):
                    s1.append(float(lista[7])/float(lista[8]))
                    
                if (float(lista[10]) > 0):
                    s1.append(float(lista[9])/float(lista[10]))
                    
                if (float(lista[12]) > 0):
                    s1.append(float(lista[11])/float(lista[12]))

            reader.close()
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            reader.readline()
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[8]) > 0):
                    s2.append(float(lista[7])/float(lista[8]))
                    
                if (float(lista[10]) > 0):
                    s2.append(float(lista[9])/float(lista[10]))
                    
                if (float(lista[12]) > 0):
                    s2.append(float(lista[11])/float(lista[12]))
                
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n') 


    writer.write('NUTRICIONISTA->*\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    ##NUTRICIONISTA -> *
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.strip().split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[14]) > 0):
                    s1.append(float(lista[13])/float(lista[14]))
                    
                if (float(lista[16]) > 0):
                    s1.append(float(lista[15])/float(lista[16]))
                    
                if (float(lista[18]) > 0):
                    s1.append(float(lista[17])/float(lista[18]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[6])
            s2 = []
            for line in reader:
                lista = line.strip().split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[14]) > 0):
                    s2.append(float(lista[13])/float(lista[14]))
                    
                if (float(lista[16]) > 0):
                    s2.append(float(lista[15])/float(lista[16]))
                    
                if (float(lista[18]) > 0):
                    s2.append(float(lista[17])/float(lista[18]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')

        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')        

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')
    
    ##* -> MEDICO
    writer.write('*->MEDICO\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0

    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            print(reader.readline().split(';')[2])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[2]) > 0):
                    s1.append(float(lista[1])/float(lista[2]))
                
                if (float(lista[8]) > 0):
                    s1.append(float(lista[7])/float(lista[8]))
                
                if (float(lista[14]) > 0):
                    s1.append(float(lista[13])/float(lista[14]))
                
                
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            reader.readline()
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[2]) > 0):
                    s2.append(float(lista[1])/float(lista[2]))
                    
                if (float(lista[8]) > 0):
                    s2.append(float(lista[7])/float(lista[8]))

                if (float(lista[14]) > 0):
                    s2.append(float(lista[13])/float(lista[14]))

            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) - 2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')


        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')
    
    
    ##* -> ENFERMERA
    writer.write('*->ENFERMERA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0

    
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            print(reader.readline().split(';')[4])
            s1 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[4]) > 0):
                    s1.append(float(lista[3])/float(lista[4]))
                    
                if (float(lista[10]) > 0):
                    s1.append(float(lista[9])/float(lista[10]))
                    
                if (float(lista[16]) > 0):
                    s1.append(float(lista[15])/float(lista[16]))

            reader.close()
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            reader.readline()
            s2 = []
            for line in reader:
                lista = line.split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[4]) > 0):
                    s2.append(float(lista[3])/float(lista[4]))
                    
                if (float(lista[10]) > 0):
                    s2.append(float(lista[9])/float(lista[10]))
                    
                if (float(lista[16]) > 0):
                    s2.append(float(lista[15])/float(lista[16]))
                
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')
        
        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n') 


    writer.write('*->NUTRICIONISTA\n;')
    for i in range(len(clusters)):
        writer.write(clusters[i])
        if i < len(clusters) - 1:
            writer.write(';')
    writer.write('\n')
    index_list = 1
    wrote_list = 0
    
    ##* -> NUTRICIONISTA
    for i in range(len(clusters)-1):
        writer.write(clusters[i] + ';' + ';'*index_list)
        for j in range(i+1, len(clusters)):
            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[i]+'.csv', 'r')
            s = reader.readline().split(';')
            #print(s)
            #print(s[6])
            s1 = []
            for line in reader:
                lista = line.strip().split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[6]) > 0):
                    s1.append(float(lista[5])/float(lista[6]))
                    
                if (float(lista[12]) > 0):
                    s1.append(float(lista[11])/float(lista[12]))
                    
                if (float(lista[18]) > 0):
                    s1.append(float(lista[17])/float(lista[18]))
            reader.close()

            reader = open('Reports/Explicit Derivation/Nuevos Clusters/PAD R '+clusters[j]+'.csv', 'r')
            s = reader.readline().split(';')
            print(s[6])
            s2 = []
            for line in reader:
                lista = line.strip().split(';')
                if lista[0] not in lista_p:
                    continue
                if (float(lista[6]) > 0):
                    s2.append(float(lista[5])/float(lista[6]))
                    
                if (float(lista[12]) > 0):
                    s2.append(float(lista[11])/float(lista[12]))
                    
                if (float(lista[18]) > 0):
                    s2.append(float(lista[17])/float(lista[18]))
            reader.close()
            if (np.mean(s1) > 0 or np.mean(s2) > 0):
                print (clusters[i] + ' V/S ' + clusters[j])
                writer.write(t_test(s1, s2, 0.95))
            else:
                writer.write('N/A')
            if j < len(clusters) - 1:
                writer.write(';')

        writer2.write(clusters[i] + ';')
        writer2.write(str(np.mean(s1))+';'+str(len(s1)))
        writer2.write('\n')
        if i == len(clusters) -2:
            writer2.write(clusters[j] + ';')
            writer2.write(str(np.mean(s2))+';'+str(len(s2)))
            writer2.write('\n')        

        writer.write('\n')
        index_list += 1
    writer.write('\n')
    writer2.write('\n')
    
    
    writer.close()
    writer2.close()

In [9]:
    #clusters = ['Delegador', 'Deleg. reasig.', 'Subcon. simple', 'Subcon. multiple', 'Subcon. coord.', "Grupo2", "Grupo Int2"]
    #clusters = ['JPII', 'MTC', 'SAH']
    clusters = ['Mejora', 'Compensado', 'Medianamente Descompensado', 'Altamente Descompensado', "Global"]
    
    #indepence_test("AdherenciaCTCV_label.csv", 0.99)
    #adherence_test(clusters)
    detailed_adherence_test(clusters)
    detailed_adherence_test_from_to(clusters)
    #normalized_explicit_test(clusters, 1)

TOTAL ('MEDICO', 'MEDICO')
Mejora V/S Compensado
Medias: 0.643583099703 0.65279958585
TOTAL ('MEDICO', 'MEDICO')
Mejora V/S Medianamente Descompensado
Medias: 0.643583099703 0.623280019757
TOTAL ('MEDICO', 'MEDICO')
Mejora V/S Altamente Descompensado
Medias: 0.643583099703 0.609110152738
TOTAL ('MEDICO', 'MEDICO')
Mejora V/S Global
Medias: 0.643583099703 0.627082833609
TOTAL ('MEDICO', 'MEDICO')
Compensado V/S Medianamente Descompensado
Medias: 0.65279958585 0.623280019757
TOTAL ('MEDICO', 'MEDICO')
Compensado V/S Altamente Descompensado
Medias: 0.65279958585 0.609110152738
TOTAL ('MEDICO', 'MEDICO')
Compensado V/S Global
Medias: 0.65279958585 0.627082833609
TOTAL ('MEDICO', 'MEDICO')
Medianamente Descompensado V/S Altamente Descompensado
Medias: 0.623280019757 0.609110152738
TOTAL ('MEDICO', 'MEDICO')
Medianamente Descompensado V/S Global
Medias: 0.623280019757 0.627082833609
TOTAL ('MEDICO', 'MEDICO')
Altamente Descompensado V/S Global
Medias: 0.609110152738 0.627082833609
TOTAL ('ME

Medianamente Descompensado V/S Global
Medias: 0.639455782313 0.630456349206
TOTAL ('NUTRICIONISTA', 'ENFERMERA')
Altamente Descompensado V/S Global
Medias: 0.581666666667 0.630456349206
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')
Mejora V/S Compensado
Medias: 0.561594202899 0.535087719298
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')
Mejora V/S Medianamente Descompensado
Medias: 0.561594202899 0.533333333333
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')
Mejora V/S Altamente Descompensado
Medias: 0.561594202899 0.383333333333
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')

Mejora V/S Global
Medias: 0.561594202899 0.466324200913
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')
Compensado V/S Medianamente Descompensado
Medias: 0.535087719298 0.533333333333
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')
Compensado V/S Altamente Descompensado
Medias: 0.535087719298 0.383333333333
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')

Compensado V/S Global
Medias: 0.535087719298 0.466324200913
TOTAL ('NUTRICIONISTA', 'NUTRICIONISTA')
Me